# Test-Charaktere
Für erste Tests erst einmal 2 simple Charaktere um die Grundlagen zu testen.\\
Später wird das per File einlesbar sein, in welchem Format muss man dann jedoch schauen.

In [122]:
import random

## Glossar

In [123]:
glossar_one = {
    "MU": "Mut",
    "KL": "Klugheit",
    "IN": "Intuition",
    "CH": "Charisma",
    "FF": "Fingerfertigkeit",
    "GE": "Gewandheit",
    "KO": "Konstitution",
    "KK": "Körperkraft",

    "LP": "Lebenspunkt",
    "AsP": "Astralpunkt",
    "AUS": "Ausdauer",
    "INIT": "Initiative",
    "AP": "Abenteurerpunkte"
}

glossar_two = {val: key for key, val in glossar_one.items()}

glossar = glossar_one | glossar_two

In [124]:
def w20():
    return random.randint(1, 20)


def w6():
    return random.randint(1, 6)


dsa3_eigenschaften = ["MU", "KL", "IN", "CH", "FF", "GE", "KO", "KK"]
dsa3_startwerte = {
    "LP": 12,
    "AsP": 12,
    "AUS": 12,
    "INIT": 12,
    "AP": 0
    }


def generate_eigenschaften(skills):
    """
    Generiert ein dictionary an random Werten (1-20) für die eigenschaften eines
    DSA Charakters. Diese dienen erst einmal nur Tetszwecken und können später
    noch verfeinert werden.
    """

    eigenschaften = {}

    for skill in skills:
        eigenschaften[skill] = random.randint(1, 20)

    return eigenschaften



def generate_steckbrief(name, alter, geschlecht, rasse, heldentyp):
    """

    """

    label = ["name", "alter", "geschlecht", "rasse", "heldentyp"]
    infos = [name, alter, geschlecht, rasse, heldentyp]

    steckbrief = dict(zip(label, infos)) | dsa3_startwerte
    
    steckbrief["eigenschaften"] = generate_eigenschaften(dsa3_eigenschaften)

    return steckbrief


In [125]:
data_parnileo_parnilei = generate_steckbrief("Parnileo Parnilei", 461, "männlich", "Mensch", "Magier")
data_edgar_allan_parn = generate_steckbrief("Edgar Allan Parn", 216, "männlich", "Mensch", "Magier")

In [130]:
class Entity():
    def __init__(self, file):
        if file is not None:
            for key, value in file.items():
                setattr(self, key, value)

    def eigenschaftsprobe(self, probe, bias=0):
        wurf = w20()

        if wurf > self.eigenschaften[glossar[probe]] + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {self.eigenschaften[glossar[probe]]} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {self.eigenschaften[glossar[probe]]} wurde durch {wurf} unterworfen!")

In [135]:
parnileo_parnilei = Entity(data_parnileo_parnilei)
edgar_allan_parn = Entity(data_edgar_allan_parn)

glossar["Klugheit"]

'KL'

In [137]:
print(f"Hallo, ich bin {parnileo_parnilei.name} und ich habe in Mut einen Wert von {parnileo_parnilei.eigenschaften['MU']}.")

parnileo_parnilei.eigenschaftsprobe("Mut")

Hallo, ich bin Parnileo Parnilei und ich habe in Mut einen Wert von 16.
Eigenschaftsprobe geschafft! Der nötige Mut-Wert von 16 wurde durch 10 unterworfen!
